# Datentransformation Bureau

Ziel dieser Datenaufbereitung ist es alle Variablen so in einen Datensatz zu überführen.

In [1]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from IPython.display import display, Markdown

In [2]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [3]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
bureau = pd.read_csv(DATASET_DIR / "bureau.csv")
bureau_balance = pd.read_csv(DATASET_DIR / "bureau_balance.csv")
des = pd.read_csv(DATASET_DIR / "HomeCredit_columns_description.csv", encoding="latin", index_col=0)

In [7]:
des.loc[des['Table']=="bureau.csv", "Row":"Special"]

,Row,Description,Special
125,SK_ID_CURR,"ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau",hashed
126,SK_BUREAU_ID,Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application),hashed
127,CREDIT_ACTIVE,Status of the Credit Bureau (CB) reported credits,NaN
128,CREDIT_CURRENCY,Recoded currency of the Credit Bureau credit,recoded
129,DAYS_CREDIT,How many days before current application did client apply for Credit Bureau credit,time only relative to the application
130,CREDIT_DAY_OVERDUE,Number of days past due on CB credit at the time of application for related loan in our sample,NaN
131,DAYS_CREDIT_ENDDATE,Remaining duration of CB credit (in days) at the time of application in Home Credit,time only relative to the application
132,DAYS_ENDDATE_FACT,Days since CB credit ended at the time of application in Home Credit (only for closed credit),time only relative to the application
133,AMT_CREDIT_MAX_OVERDUE,Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample),NaN
134,CNT_CREDIT_PROLONG,How many times was the Credit Bureau credit prolonged,NaN


In [4]:
todo = list(bureau.columns)

#### Keine Datenaufbereitung für Primär- und Fremdschlüssel

In [5]:
keys = ["SK_ID_CURR", "SK_ID_BUREAU"]

In [6]:
todo = [element for element in todo if element not in keys]

#### Datenaufbereitung metrische Daten

In [7]:
m_heads = [element for element in todo if bureau[element].dtype.name != "object"]

##### Gruppierung

In [8]:
quants = [0, 0.25, 0.5, 0.75, 1]

In [17]:
df = bureau[["SK_ID_CURR", m_heads[0]]]
df = df.groupby(by = ["SK_ID_CURR"]).quantile(quants)
df = df.unstack(level=-1)

In [ ]:
pd.concat({'Foo': df}, names=['Firstlevel'])

In [ ]:
df.head()

In [9]:
for index, head in enumerate(m_heads):
    
    df = bureau[["SK_ID_CURR", head]]
    df = df.groupby(by = ["SK_ID_CURR"]).quantile(quants)
    df = df.unstack(level=-1)
    
    try:
        mets = mets.join(df , on = "SK_ID_CURR")
    except (ValueError, NameError):
        mets = df

In [10]:
# Löschen der metrischen Daten aus der TODO Liste
todo = [element for element in todo if element not in m_heads]

In [11]:
mets.head()

DAYS_CREDIT                                CREDIT_DAY_OVERDUE       \
                  0.00    0.25    0.50    0.75   1.00               0.00 0.25   
SK_ID_CURR                                                                      
100001         -1572.0  -894.0  -857.0 -439.50  -49.0                0.0  0.0   
100002         -1437.0 -1122.0 -1042.5 -602.75 -103.0                0.0  0.0   
100003         -2586.0 -1873.5 -1205.5 -732.75 -606.0                0.0  0.0   
100004         -1326.0 -1096.5  -867.0 -637.50 -408.0                0.0  0.0   
100005          -373.0  -255.0  -137.0  -99.50  -62.0                0.0  0.0   

                          DAYS_CREDIT_ENDDATE                                  \
           0.50 0.75 1.00                0.00     0.25   0.50    0.75    1.00   
SK_ID_CURR                                                                      
100001      0.0  0.0  0.0             -1329.0  -503.00 -179.0  656.50  1778.0   
100002      0.0  0.0  0.0             -1072.0 -1006.25 -424.5   79.25   780.0   
100003      0.0  0.0  0.0             -2434.0 -1013.50 -480.0  -11.00  1216.0   
100004      0.0  0.0  0.0              -595.0  -541.75 -488.5 -435.25  -382.0   
100005      0.0  0.0  0.0              -128.0    -3.00  122.0  723.00  1324.0   

           DAYS_ENDDATE_FACT                                 \
                        0.00     0.25   0.50    0.75   1.00   
SK_ID_CURR                                                    
100001               -1328.0  -989.75 -715.0 -550.75 -544.0   
100002               -1185.0 -1020.25 -939.0 -263.75  -36.0   
100003               -2131.0 -1376.00 -621.0 -580.50 -540.0   
100004                -683.0  -607.75 -532.5 -457.25 -382.0   
100005                -123.0  -123.00 -123.0 -123.00 -123.0   

           AMT_CREDIT_MAX_OVERDUE                               \
                             0.00 0.25  0.50    0.75      1.00   
SK_ID_CURR                                                       
100001                        NaN  NaN   NaN     NaN       NaN   
100002                        0.0  0.0  40.5  3321.0  5043.645   
100003                        0.0  0.0   0.0     0.0     0.000   
100004                        0.0  0.0   0.0     0.0     0.000   
100005                        0.0  0.0   0.0     0.0     0.000   

           CNT_CREDIT_PROLONG                     AMT_CREDIT_SUM  \
                         0.00 0.25 0.50 0.75 1.00           0.00   
SK_ID_CURR                                                         
100001                    0.0  0.0  0.0  0.0  0.0        85500.0   
100002                    0.0  0.0  0.0  0.0  0.0            0.0   
100003                    0.0  0.0  0.0  0.0  0.0        22248.0   
100004                    0.0  0.0  0.0  0.0  0.0        94500.0   
100005                    0.0  0.0  0.0  0.0  0.0        29826.0   

                                                         AMT_CREDIT_SUM_DEBT  \
                    0.25       0.50       0.75      1.00                0.00   
SK_ID_CURR                                                                     
100001      102060.00000  168345.00  308700.00  378000.0                 0.0   
100002       28759.17375   54130.50  124301.25  450000.0                 0.0   
100003       60051.37500   92576.25  286875.00  810000.0                 0.0   
100004       94509.45000   94518.90   94528.35   94537.8                 0.0   
100005       44163.00000   58500.00  313650.00  568800.0                 0.0   

                                                   AMT_CREDIT_SUM_LIMIT       \
                0.25     0.50       0.75      1.00                 0.00 0.25   
SK_ID_CURR                                                                     
100001          0.00      0.0  111723.75  373239.0                  0.0  0.0   
100002          0.00      0.0       0.00  245781.0                  0.0  0.0   
100003          0.00      0.0       0.00       0.0                  0.0  0.0   
100004       

#### Datenaufbereitung kategorische Daten

In [12]:
cat_heads = [element for element in todo if bureau[element].dtype.name == "object"]

In [13]:
for index, head in enumerate(cat_heads):
    
    df = bureau[["SK_ID_CURR", head]]
    df = df.groupby(["SK_ID_CURR", head]).size()
    df = df.unstack()
    
    try:
        cats = cats.join(a , on="SK_ID_CURR")
    except (ValueError, NameError):
        cats = df

In [14]:
# Löschen der metrischen Daten aus der TODO Liste
todo = [element for element in todo if element not in cat_heads]

In [15]:
cats.head()

CREDIT_TYPE,Another type of loan,Car loan,Cash loan (non-earmarked),Consumer credit,Credit card,Interbank credit,Loan for business development,Loan for purchase of shares (margin lending),Loan for the purchase of equipment,Loan for working capital replenishment,Microloan,Mobile operator loan,Mortgage,Real estate loan,Unknown type of loan
SK_ID_CURR,,,,,,,,,,,,,,,
100001,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,NaN,NaN,NaN,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Zusammenführung metrischer und kategorischer Daten

In [16]:
bureau = mets.join(cats, on="SK_ID_CURR")

c:\users\rober\documents\workspace\python\masterarbeit\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


In [18]:
bureau.head()

,"(DAYS_CREDIT, 0.0)","(DAYS_CREDIT, 0.25)","(DAYS_CREDIT, 0.5)","(DAYS_CREDIT, 0.75)","(DAYS_CREDIT, 1.0)","(CREDIT_DAY_OVERDUE, 0.0)","(CREDIT_DAY_OVERDUE, 0.25)","(CREDIT_DAY_OVERDUE, 0.5)","(CREDIT_DAY_OVERDUE, 0.75)","(CREDIT_DAY_OVERDUE, 1.0)","(DAYS_CREDIT_ENDDATE, 0.0)","(DAYS_CREDIT_ENDDATE, 0.25)","(DAYS_CREDIT_ENDDATE, 0.5)","(DAYS_CREDIT_ENDDATE, 0.75)","(DAYS_CREDIT_ENDDATE, 1.0)","(DAYS_ENDDATE_FACT, 0.0)","(DAYS_ENDDATE_FACT, 0.25)","(DAYS_ENDDATE_FACT, 0.5)","(DAYS_ENDDATE_FACT, 0.75)","(DAYS_ENDDATE_FACT, 1.0)","(AMT_CREDIT_MAX_OVERDUE, 0.0)","(AMT_CREDIT_MAX_OVERDUE, 0.25)","(AMT_CREDIT_MAX_OVERDUE, 0.5)","(AMT_CREDIT_MAX_OVERDUE, 0.75)","(AMT_CREDIT_MAX_OVERDUE, 1.0)","(CNT_CREDIT_PROLONG, 0.0)","(CNT_CREDIT_PROLONG, 0.25)","(CNT_CREDIT_PROLONG, 0.5)","(CNT_CREDIT_PROLONG, 0.75)","(CNT_CREDIT_PROLONG, 1.0)","(AMT_CREDIT_SUM, 0.0)","(AMT_CREDIT_SUM, 0.25)","(AMT_CREDIT_SUM, 0.5)","(AMT_CREDIT_SUM, 0.75)","(AMT_CREDIT_SUM, 1.0)","(AMT_CREDIT_SUM_DEBT, 0.0)","(AMT_CREDIT_SUM_DEBT, 0.25)","(AMT_CREDIT_SUM_DEBT, 0.5)","(AMT_CREDIT_SUM_DEBT, 0.75)","(AMT_CREDIT_SUM_DEBT, 1.0)","(AMT_CREDIT_SUM_LIMIT, 0.0)","(AMT_CREDIT_SUM_LIMIT, 0.25)","(AMT_CREDIT_SUM_LIMIT, 0.5)","(AMT_CREDIT_SUM_LIMIT, 0.75)","(AMT_CREDIT_SUM_LIMIT, 1.0)","(AMT_CREDIT_SUM_OVERDUE, 0.0)","(AMT_CREDIT_SUM_OVERDUE, 0.25)","(AMT_CREDIT_SUM_OVERDUE, 0.5)","(AMT_CREDIT_SUM_OVERDUE, 0.75)","(AMT_CREDIT_SUM_OVERDUE, 1.0)","(DAYS_CREDIT_UPDATE, 0.0)","(DAYS_CREDIT_UPDATE, 0.25)","(DAYS_CREDIT_UPDATE, 0.5)","(DAYS_CREDIT_UPDATE, 0.75)","(DAYS_CREDIT_UPDATE, 1.0)","(AMT_ANNUITY, 0.0)","(AMT_ANNUITY, 0.25)","(AMT_ANNUITY, 0.5)","(AMT_ANNUITY, 0.75)","(AMT_ANNUITY, 1.0)",Another type of loan,Car loan,Cash loan (non-earmarked),Consumer credit,Credit card,Interbank credit,Loan for business development,Loan for purchase of shares (margin lending),Loan for the purchase of equipment,Loan for working capital replenishment,Microloan,Mobile operator loan,Mortgage,Real estate loan,Unknown type of loan
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100001,-1572.0,-894.0,-857.0,-439.50,-49.0,0.0,0.0,0.0,0.0,0.0,-1329.0,-503.00,-179.0,656.50,1778.0,-1328.0,-989.75,-715.0,-550.75,-544.0,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,85500.0,102060.00000,168345.00,308700.00,378000.0,0.0,0.00,0.0,111723.75,373239.0,0.0,0.0,0.0,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,-155.0,-155.00,-155.0,-13.00,-6.0,0.0,0.0,0.0,6997.50,10822.5,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100002,-1437.0,-1122.0,-1042.5,-602.75,-103.0,0.0,0.0,0.0,0.0,0.0,-1072.0,-1006.25,-424.5,79.25,780.0,-1185.0,-1020.25,-939.0,-263.75,-36.0,0.0,0.0,40.5,3321.0,5043.645,0.0,0.0,0.0,0.0,0.0,0.0,28759.17375,54130.50,124301.25,450000.0,0.0,0.00,0.0,0.00,245781.0,0.0,0.0,0.0,7997.14125,31988.565,0.0,0.0,0.0,0.0,0.0,-1185.0,-939.00,-402.5,-31.50,-7.0,0.0,0.0,0.0,0.00,0.0,NaN,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100003,-2586.0,-1873.5,-1205.5,-732.75,-606.0,0.0,0.0,0.0,0.0,0.0,-2434.0,-1013.50,-480.0,-11.00,1216.0,-2131.0,-1376.00,-621.0,-580.50,-540.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,22248.0,60051.37500,92576.25,286875.00,810000.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,202500.00000,810000.000,0.0,0.0,0.0,0.0,0.0,-2131.0,-945.25,-545.0,-415.75,-43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100004,-1326.0,-1096.5,-867.0,-637.50,-408.0,0.0,0.0,0.0,0.0,0.0,-595.0,-541.75,-488.5,-435.25,-382.0,-683.0,-607.75,-532.5,-457.25,-382.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,94500.0,94509.45000,94518.90,94528.35,94537.8,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.00000,0.000,0.0,0.0,0.0,0.0,0.0,-682.0,-607.00,-532.0,-457.00,-382.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
100005,-373.0,-255.0,-137.0,-99.50,-62.0,0.0,0.0,0.0,0.0,0.0,-128.0,-3.00,122.0,723.00,1324.0,-123.0,-123.00,-123.0,-123.00,-123.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,29826.0,44163.00000,58500.00,313650.00,568800.

### Abspeichern der Daten

In [19]:
bureau.to_csv(DATASET_DIR / "Datengruppierung" / "bureau.csv")

In [20]:
todo

[]